In [ ]:
# 查看模型的层数
print(f"ResNet50 层数: {len(resnet.layers)}")
print(f"Xception 层数: {len(xception.layers)}")
print(f"InceptionV3 层数: {len(inception.layers)}")


# 加载权重并查看层数
resnet.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\resnet50-imagenet-finetune152.h5", by_name=True)
print(f"加载后的 ResNet50 层数: {len(resnet.layers)}")

xception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\xception-imagenet-finetune116.h5", by_name=True)
print(f"加载后的 Xception 层数: {len(xception.layers)}")

inception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\inceptionV3-imagenet-finetune172.h5", by_name=True)
print(f"加载后的 InceptionV3 层数: {len(inception.layers)}")


In [3]:
import numpy as np
import tensorflow as tf

# 使用tensorflow下的keras库是错误的，暂时不知道原因
# from tensorflow.keras.applications import ResNet50, Xception, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xcep_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as incep_preprocess
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras.models import load_model
# from tensorflow.keras import layers, models

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

# 设置模型输入尺寸（统一为224，以兼容所有模型）
image_size = 224  # Xception 原本用 299，可以调整为 224

# 加载模型并设置权重路径
resnet = ResNet50(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
xception = Xception(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
inception = InceptionV3(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))

# 加载权重
resnet.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\resnet50-imagenet-finetune152.h5", by_name=True)
xception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\xception-imagenet-finetune116.h5", by_name=True)
inception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\inceptionV3-imagenet-finetune172.h5", by_name=True)

# 加载你训练好的融合分类器
model_mix = load_model(r"F:\JupyterWorkSpace\BBBBBBS\models\mixed-model.h5")

def predict_single_image(image_path):
    # 加载图像并调整为模型输入尺寸
    img = load_img(image_path, target_size=(image_size, image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # 使用三种模型提取特征（各自预处理）
    feat_resnet = resnet.predict(resnet_preprocess(np.copy(img_array)))
    feat_xcep = xception.predict(xcep_preprocess(np.copy(img_array)))
    feat_incep = inception.predict(incep_preprocess(np.copy(img_array)))

    # 拼接所有模型的 bottleneck 特征
    bottleneck_feature = np.concatenate([feat_resnet, feat_xcep, feat_incep], axis=1)

    # 用混合分类器做最终预测
    prediction = model_mix.predict(bottleneck_feature)

    # 输出预测结果
    print("预测概率分布：", prediction[0])
    class_idx = np.argmax(prediction[0])
    classes = ['c0安全驾驶', 'c1右手打字', 'c2右手接电话','c3左手打字','c4左手接电话',
               'c5调收音机','c6喝饮料','c7拿后面的东西','c8整理头发和化妆','c9和其他乘客说话']
    print("预测类别索引：", class_idx)
    print("概率：",prediction[0][class_idx])
    print("预测类别名称：", classes[class_idx])


In [4]:
# 测试
image_path = r"F:\BBBBBBBBBBBBBBBBB\Datas\imgs\train\c5\img_735.jpg"
predict_single_image(image_path)

1/1 [==============================] - 0s 36ms/step
预测概率分布： [0.06741077 0.04833904 0.11561967 0.09600563 0.10461302 0.30650067
 0.01295456 0.21692894 0.01605576 0.01557183]
预测类别索引： 5
概率： 0.30650067
预测类别名称： c5调收音机


### 这里又可以使用tensorflow的keras了，并且实现了简单的图形化界面

In [38]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xcep_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as incep_preprocess
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk

# 设置模型输入尺寸（统一为224，以兼容所有模型）
image_size = 224  # Xception 原本用 299，可以调整为 224

# 加载模型并设置权重路径
resnet = ResNet50(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
xception = Xception(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))
inception = InceptionV3(weights=None, include_top=False, pooling='avg', input_shape=(image_size, image_size, 3))

# 加载权重
resnet.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\resnet50-imagenet-finetune152.h5", by_name=True)
xception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\xception-imagenet-finetune116.h5", by_name=True)
inception.load_weights(r"F:\JupyterWorkSpace\BBBBBBS\models\inceptionV3-imagenet-finetune172.h5", by_name=True)

# 加载你训练好的融合分类器
model_mix = load_model(r"F:\JupyterWorkSpace\BBBBBBS\models\mixed-model.h5")

# 定义预测函数
def predict_single_image(image_path):
    # 加载图像并调整为模型输入尺寸
    img = load_img(image_path, target_size=(image_size, image_size))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # 使用三种模型提取特征（各自预处理）
    feat_resnet = resnet.predict(resnet_preprocess(np.copy(img_array)))
    feat_xcep = xception.predict(xcep_preprocess(np.copy(img_array)))
    feat_incep = inception.predict(incep_preprocess(np.copy(img_array)))

    # 拼接所有模型的 bottleneck 特征
    bottleneck_feature = np.concatenate([feat_resnet, feat_xcep, feat_incep], axis=1)

    # 用混合分类器做最终预测
    prediction = model_mix.predict(bottleneck_feature)

    # 获取预测结果并显示
    class_idx = np.argmax(prediction[0])
    classes = ['c0安全驾驶', 'c1右手打字', 'c2右手接电话', 'c3左手打字', 'c4左手接电话',
               'c5调收音机', 'c6喝饮料', 'c7拿后面的东西', 'c8整理头发和化妆', 'c9和其他乘客说话']
    predicted_class = classes[class_idx]
    predicted_prob = prediction[0][class_idx]
    return predicted_class, predicted_prob, image_path

# 定义图形界面
def open_image():
    file_path = filedialog.askopenfilename(title="选择图片", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predicted_class, predicted_prob, image_path = predict_single_image(file_path)
        
        # 显示图像
        img = Image.open(image_path)
        img.thumbnail((200, 200))
        img = ImageTk.PhotoImage(img)
        panel_img.config(image=img)
        panel_img.image = img
        
        # 显示预测结果
        result_text = f"预测类别: {predicted_class}\n预测概率: {predicted_prob:.4f}"
        label_result.config(text=result_text)

        # 显示文件路径
        label_file_path.config(text=f"文件路径: {image_path}")

# 创建窗口
root = tk.Tk()
root.config(bg="#f4e1f1")  # 或者 root = tk.Tk(bg="lightblue")
root.title("图像分类预测")
root.geometry("800x600")# 宽*高
# root.configure(bg="#f0f0f0")

# 设置字体样式
font_style = ("Arial", 12)

# 创建按钮和标签
btn_open_image = tk.Button(root, text="选择图片", command=open_image, width=20, height=2, bg="#e0f7fa", fg="black", font=font_style)
btn_open_image.pack(pady=20)

panel_img = tk.Label(root, bg="#e0f7fa")
panel_img.pack(pady=10)

label_result = tk.Label(root, text="预测结果", font=("Arial", 14, "bold"), justify="left", width=30, height=4, bg="#e0f7fa")
label_result.pack(pady=10)

label_file_path = tk.Label(root, text="文件路径", font=font_style, width=80, height=2, bg="#e0f7fa")
label_file_path.pack(pady=10)

# 启动界面
root.mainloop()
